In [1]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
from neo4j import GraphDatabase
from transformers import pipeline

# Initialize the model using Ollama
model = OllamaLLM(model="llama3.1")
prompt = ChatPromptTemplate.from_template("""
Answer the question below using available information.

Here is the conversation history: {context}

Relevant Information: {knowledge_graph_data}

Question: {question}

Answer:
""")
chain = prompt | model

# Initialize Memory Module
memory = ConversationBufferMemory()

# Set up VectorDB for embedding storage and retrieval
dimension = 384  # Dimension for Sentence-BERT embeddings (e.g., all-MiniLM-L6-v2)
index = faiss.IndexFlatL2(dimension)  # Initialize FAISS index for similarity search

# Initialize the Sentence-BERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to Neo4j
uri = "neo4j+s://a00a22f6.databases.neo4j.io"  
username = ""  # Replace with your Neo4j username
password = ""  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

def close_driver():
    driver.close()
    
def generate_embedding(text):
    # Generate embedding using the Sentence-BERT model
    embedding = sbert_model.encode(text)
    return embedding

def add_to_memory(user_input, ai_response, embedding):
    # Save both user input and AI response in memory buffer
    memory.save_context({"question": user_input}, {"answer": ai_response})
    # Add the embedding to the FAISS index
    index.add(np.array([embedding]))

def retrieve_from_memory(query_embedding):
    if index.ntotal == 0:
        return memory.buffer  # Return the whole conversation history if no embeddings are stored
    
    # Retrieve the closest context based on cosine similarity
    D, I = index.search(np.array([query_embedding]), k=1)
    if I[0][0] != -1:  # Check if a similar context was found
        return memory.buffer  # Return the whole conversation history for simplicity
    return ""

# Initialize the NER pipeline using Pre Trained Bert
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

def extract_entities(user_input):
    """
    Extract entities from user input using a pre-trained NER model.
    """
    # Run the NER model on the input text
    ner_results = ner_pipeline(user_input)
    
    # Extract the recognized entities
    entities = [entity['word'] for entity in ner_results]
    
    # Filter out duplicate entities (if any)
    unique_entities = list(set(entities))
    
    return unique_entities

def format_knowledge_graph_data(data):
    # Convert the retrieved data into a string format for the LLM
    formatted_data = "\n".join([f"{item['n']['name']} -[{item['r']['type']}]-> {item['m']['name']}" for item in data])
    return formatted_data

def calculate_relevance(user_input, data):
    # Convert user input to embedding
    user_embedding = sbert_model.encode(user_input, convert_to_tensor=True)
    
    relevant_data = []
    for item in data:
        # Combine the textual content of nodes and relationships
        text_content = f"{item['n']['name']} {item['m']['name']} {item['r']['type']}"
        
        # Convert the knowledge graph text to embedding
        kg_embedding = sbert_model.encode(text_content, convert_to_tensor=True)
        
        # Calculate cosine similarity
        similarity = util.pytorch_cos_sim(user_embedding, kg_embedding)
        
        # Store only relevant results 
        if similarity.item() > 0.5:
            relevant_data.append((similarity.item(), text_content))
    
    # Sort by relevance
    relevant_data.sort(key=lambda x: x[0], reverse=True)
    return relevant_data

def adaptive_depth_search(user_input, initial_depth=1, max_depth=2):
    """
    Start with an initial depth and adaptively increase it based on relevance.
    """
    for depth in range(initial_depth, max_depth + 1):
        # Step 1: Extract entities
        entities = extract_entities(user_input)
        
        # Step 2: Query knowledge graph with current depth
        entity_placeholders = ', '.join(f"'{entity}'" for entity in entities)
        query = f"""
        MATCH (n)-[r*1..{depth}]->(m)
        WHERE n.name IN [{entity_placeholders}] OR m.name IN [{entity_placeholders}]
        RETURN n, r, m
        LIMIT 100
        """
        with driver.session() as session:
            result = session.run(query)
            raw_data = [record.data() for record in result]
        
        # Step 3: Calculate the relevance of retrieved data to the user input
        relevant_data = calculate_relevance(user_input, raw_data)
        
        # If relevant data is found, return it
        if relevant_data:
            return relevant_data
        
    return []  # Return an empty list if no relevant data is found

C:\Users\hanta\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing

In [2]:
def handle_conversation():
    print("Welcome to the AI Chatbot! Type 'exit' to quit.")
    context = ""
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        
        # Step 1: Generate embedding for the current input
        prompt_embedding = generate_embedding(user_input)
        
        # Step 2: Retrieve relevant context using the embedding
        context = retrieve_from_memory(prompt_embedding)
        
        # Step 3: Perform adaptive depth search in the knowledge graph
        relevant_data = adaptive_depth_search(user_input)
        
        # Step 4: Format the relevant knowledge graph data for the LLM
        knowledge_graph_data = format_knowledge_graph_data([data for _, data in relevant_data])
        
        # Step 5: Generate response using LLM, the retrieved context, and knowledge graph data
        result = chain.invoke({
            "context": context,
            "question": user_input,
            "knowledge_graph_data": knowledge_graph_data
        })
        print(f"Bot: {result}")
        
        # Add the current conversation turn to memory
        add_to_memory(user_input, result, prompt_embedding)
        context += f"User: {user_input}\nAI: {result}\n"  # Update the local context for continuity

if __name__ == "__main__":
    handle_conversation()
    close_driver()

Welcome to the AI Chatbot! Type 'exit' to quit.
You: Based on Gartner's recommendations, how should I establish an enterprise architecture program. Provide specific recommendations by Gartner.
Bot: Based on my knowledge of Gartner's recommendations, here are some specific steps you can take to establish an effective enterprise architecture (EA) program:

1. **Develop a clear EA vision and strategy**: According to Gartner, the first step in establishing an EA program is to develop a clear vision and strategy that aligns with your organization's business objectives [1]. This should include defining the role of EA within the organization, its scope, and key deliverables.
2. **Establish a Center of Excellence (COE)**: Gartner recommends establishing an Enterprise Architecture COE (EA COE) to oversee the development and implementation of EA across the organization [2]. The EA COE should be responsible for defining and maintaining the enterprise architecture framework, standards, and guideli

# Simplified Depth Search Version

In [3]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
from neo4j import GraphDatabase
from transformers import pipeline

# Initialize the model using Ollama
model = OllamaLLM(model="llama3.1")
prompt = ChatPromptTemplate.from_template("""
Answer the question below using available information.

Here is the conversation history: {context}

Relevant Information: {knowledge_graph_data}

Question: {question}

Answer:
""")
chain = prompt | model

# Initialize Memory Module
memory = ConversationBufferMemory()

# Set up VectorDB for embedding storage and retrieval
dimension = 384  # Dimension for Sentence-BERT embeddings (e.g., all-MiniLM-L6-v2)
index = faiss.IndexFlatL2(dimension)  # Initialize FAISS index for similarity search

# Initialize the Sentence-BERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to Neo4j
uri = "neo4j+s://a00a22f6.databases.neo4j.io"  
username = ""  # Replace with your Neo4j username
password = ""  # Replace with your Neo4j password
driver = GraphDatabase.driver(uri, auth=(username, password))

def close_driver():
    driver.close()
    
def generate_embedding(text):
    # Generate embedding using the Sentence-BERT model
    embedding = sbert_model.encode(text)
    return embedding

def add_to_memory(user_input, ai_response, embedding):
    # Save both user input and AI response in memory buffer
    memory.save_context({"question": user_input}, {"answer": ai_response})
    # Add the embedding to the FAISS index
    index.add(np.array([embedding]))

def retrieve_from_memory(query_embedding):
    if index.ntotal == 0:
        return memory.buffer  # Return the whole conversation history if no embeddings are stored
    
    # Retrieve the closest context based on cosine similarity
    D, I = index.search(np.array([query_embedding]), k=1)
    if I[0][0] != -1:  # Check if a similar context was found
        return memory.buffer  # Return the whole conversation history for simplicity
    return ""

# Initialize the NER pipeline using Pre Trained Bert
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", aggregation_strategy="simple")

def extract_entities(user_input):
    """
    Extract entities from user input using a pre-trained NER model.
    """
    # Run the NER model on the input text
    ner_results = ner_pipeline(user_input)
    
    # Extract the recognized entities
    entities = [entity['word'] for entity in ner_results]
    
    # Filter out duplicate entities (if any)
    unique_entities = list(set(entities))
    
    return unique_entities
def query_knowledge_graph(entities):
    # Build a dynamic Cypher query based on extracted entities
    entity_placeholders = ', '.join(f"'{entity}'" for entity in entities)
    query = f"""
    MATCH (n)-[r]->(m)
    WHERE n.name IN [{entity_placeholders}] OR m.name IN [{entity_placeholders}]
    RETURN n, r, m
    LIMIT 100
    """
    with driver.session() as session:
        result = session.run(query)
        data = [record.data() for record in result]
    return data

def format_knowledge_graph_data(data):
    # Convert the retrieved data into a string format for the LLM
    formatted_data = "\n".join([f"{item['n']['name']} -[{item['r']['type']}]-> {item['m']['name']}" for item in data])
    return formatted_data

def calculate_relevance(user_input, data):
    # Convert user input to embedding
    user_embedding = sbert_model.encode(user_input, convert_to_tensor=True)
    
    relevant_data = []
    for item in data:
        # Combine the textual content of nodes and relationships
        text_content = f"{item['n']['name']} {item['m']['name']} {item['r']['type']}"
        
        # Convert the knowledge graph text to embedding
        kg_embedding = sbert_model.encode(text_content, convert_to_tensor=True)
        
        # Calculate cosine similarity
        similarity = util.pytorch_cos_sim(user_embedding, kg_embedding)
        
        # Store only relevant results 
        if similarity.item() > 0.5:
            relevant_data.append((similarity.item(), text_content))
    
    # Sort by relevance
    relevant_data.sort(key=lambda x: x[0], reverse=True)
    return relevant_data

C:\Users\hanta\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing

In [4]:
def handle_conversation():
    print("Welcome to the AI Chatbot! Type 'exit' to quit.")
    context = ""
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        
        # Step 1: Generate embedding for the current input
        prompt_embedding = generate_embedding(user_input)
        
        # Step 2: Retrieve relevant context using the embedding
        context = retrieve_from_memory(prompt_embedding)
        
        # Step 3: Extract entities from user input
        entities = extract_entities(user_input)
        
        # Step 4: Query knowledge graph with extracted entities
        raw_data = query_knowledge_graph(entities)
        
        # Step 5: Calculate the relevance of retrieved data to the user input
        relevant_data = calculate_relevance(user_input, raw_data)
        
        # Step 6: Format the relevant knowledge graph data for the LLM
        knowledge_graph_data = format_knowledge_graph_data([data for _, data in relevant_data])
        
        # Step 7: Generate response using LLM, the retrieved context, and knowledge graph data
        result = chain.invoke({
            "context": context,
            "question": user_input,
            "knowledge_graph_data": knowledge_graph_data
        })
        print(f"Bot: {result}")
        
        # Add the current conversation turn to memory
        add_to_memory(user_input, result, prompt_embedding)
        context += f"User: {user_input}\nAI: {result}\n"  # Update the local context for continuity

if __name__ == "__main__":
    handle_conversation()
    close_driver()

Welcome to the AI Chatbot! Type 'exit' to quit.
You: Can you give me a strategy for implementing SASE and SSE at my organization? I'm interested in following the approach advocated by Gartner or Forrester.
Bot: A very timely and relevant question!

As both Gartner and Forrester have published research on Secure Access Service Edge (SASE) and Software-Defined Perimeter (SDP), which is often used interchangeably with SSE, I'll provide a strategy based on their recommended approaches.

**Gartner's SASE Approach:**

According to Gartner's research, "SASE is the convergence of network security functions (NSFs) and SD-WAN into a single, cloud-native offering." To implement SASE at your organization, consider the following steps:

1. **Assess current security posture**: Evaluate your organization's existing security infrastructure, including firewalls, intrusion detection/prevention systems, and access control.
2. **Define SASE requirements**: Identify the specific use cases and applications 